In [1]:
# wrangling
import numpy as np
import pandas as pd
import feather
pd.options.display.max_columns = 200

# preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit

# model
from keras.models import Sequential  
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping
import tensorflow as tf
from keras import backend as K

# metrics
from sklearn.metrics import accuracy_score, log_loss

# others
import os
import random as rn
from datetime import datetime
from tqdm import tqdm_notebook as tqdm
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


# load data

In [2]:
def load_rnn_data(path, window, predict_ts, isdim3=True, geo_col=["geoid10_tract"], y_cols=["crime"]):
    """
    y_cols: ["crime"] or ["incident_type_0", "incident_type_1", "incident_type_2"]
    geo_col: ["geoid10_tract"] or ["geoid10_block"]
    return y_all and x_all of given path
    """
    # load data
    df = feather.read_dataframe(path)
    df.sort_values(by=["datetime", "geoid10_tract"], inplace=True)
    df.set_index("datetime", inplace=True)

    # input columns
    x_cols = list(df.drop(y_cols + geo_col, axis=1).columns)

    # group by geoid
    geo_grs = df.groupby(by=geo_col)

    # arrayes to store x and y
    # (no of timesteps, window size, no of tracts,  no of features, )
    n_timesteps = int(len(df) / len(geo_grs)) - window - predict_ts + 1
    x_all = np.empty(shape=(n_timesteps, window, len(geo_grs), len(x_cols + y_cols)))

    # (output size, no of tracts, no of outputs)
    y_all = np.empty(shape=(n_timesteps, len(geo_grs), len(y_cols)))

    # to store geo_ids and y_all's datetime
    geo_ids = []

    y_datetime = df.index.unique()[window + predict_ts - 1:]

    for i, (geo_id, gr) in enumerate(tqdm(geo_grs)):
        geo_ids.append(geo_id)
        x_values = gr[y_cols + x_cols].values
        y_values = gr[y_cols].values

        for j in range(window, len(gr) - predict_ts + 1):
            # generate x_all
            x_all[j - window, :, i, :] = x_values[j - window:j, :]
            y_all[j - window, i, :] = y_values[j + predict_ts - 1, :]

    if isdim3:
        x_all = np.reshape(x_all,
                           newshape=(x_all.shape[0], x_all.shape[1], x_all.shape[2] * x_all.shape[3]))
        y_all = np.reshape(y_all,
                           newshape=(y_all.shape[0], y_all.shape[1] * y_all.shape[2]))

    return x_all, y_all, geo_ids, y_datetime

In [3]:
# set configuration
path = "./features/features_binary_tract_2H.feather"
window = 12
predict_ts = 1  # how many timesteps future does the model predict? 

In [4]:
# load x and y
x_all, y_all, geo_ids, y_datetime = load_rnn_data(path=path,
                                                  window=window,
                                                  predict_ts=predict_ts,
                                                  isdim3=True,
                                                  geo_col=["geoid10_tract"],
                                                  y_cols=["crime"])

In [5]:
print(x_all.shape)
print(y_all.shape)
print(len(geo_ids))  # to convert model output later
print(len(y_datetime))  # to convert model output later

(18600, 12, 1560)
(18600, 195)
195
18600


# preprocessing

In [6]:
# # scaling
# scaler = MinMaxScaler()
# x_all = scaler.fit_transform(x_all)

# modeling

In [7]:
def time_series_cv(x_all, y_all, n_splits=5, model=None, fit_params=None, baseline=False):
    """
    :param baseline: True or False (defualt: False)
    :return: train and test scores and prediction of y on test data
    """

    # prepare dictionary to store scores
    train_scores = {}
    metrics = ["acc", "log_loss"]
    for metric in metrics:
        train_scores[metric] = []
    test_scores = deepcopy(train_scores)

    # prepare dictionary to store predictions
    y_test_probs = np.zeros_like(y_all)

    # time series split
    tss = TimeSeriesSplit(n_splits=n_splits)

    for split, (train_idx, test_idx) in enumerate(tss.split(x_all, y_all)):

        print("---------- split {0} ----------".format(split))
        print("[{0:%H:%M:%S}] train_index:{1}~{2} test_index:{3}~{4}".format(
            datetime.now(), train_idx[0], train_idx[-1], test_idx[0], test_idx[-1]))

        # create train and test set
        x_train = x_all[:train_idx[-1]]
        y_train = y_all[:train_idx[-1]]
        x_test = x_all[test_idx[0]:test_idx[-1]]
        y_test = y_all[test_idx[0]:test_idx[-1]]

        if baseline:
            # return 0 for all predicted probabiliby
            y_train_prob = np.zeros_like(y_train)
            y_test_prob = np.zeros_like(y_test)

            # return 0 for all binary predictions
            y_train_pred = np.zeros_like(y_train)
            y_test_pred = np.zeros_like(y_test)

        else:            
            # train
            model.fit(x_train, y_train, **fit_params)

            # predict
            y_train_prob = model.predict(x_train)
            y_test_prob = model.predict(x_test)

            # convert form probability to binary
            y_train_pred = np.fix(y_train_prob)
            y_test_pred = np.fix(y_test_prob)

        # store test prediction
        y_test_probs[test_idx[0]:test_idx[-1]] = y_test_prob

        # calculate metrics
        train_log_loss = log_loss(y_train.flatten(), y_train_prob.flatten())
        test_log_loss = log_loss(y_test.flatten(), y_test_prob.flatten())
        train_acc = accuracy_score(y_train.flatten(), y_train_pred.flatten())
        test_acc = accuracy_score(y_test.flatten(), y_test_pred.flatten())

        # store scores
        train_scores["log_loss"].append(train_log_loss)
        test_scores["log_loss"].append(test_log_loss)
        train_scores["acc"].append(train_acc)
        test_scores["acc"].append(test_acc)

        print("[{0:%H:%M:%S}] train_log_loss:{1} test_log_loss:{2}".format(
            datetime.now(), train_log_loss, test_log_loss))
        print("[{0:%H:%M:%S}] train_acc:{1} test_acc:{2}\n".format(
            datetime.now(), train_acc, test_acc))

        # convert to dataframe
        train_scores_df = pd.DataFrame(train_scores)
        test_scores_df = pd.DataFrame(test_scores)

    return train_scores_df, test_scores_df, y_test_probs

In [8]:
class MyLSTM():
    """
    class to input into function time_series_cv
    """
    def __init__(self, units=100,
                 dropout_rate=0.2, activation="sigmoid",
                 optimizer="adam", loss="binary_crossentropy"):
        """
        define LSTM model with given arguments
        """
        self.units = units
        self.dropout_rate = dropout_rate
        self.activation = activation
        self.optimizer = optimizer
        self.loss = loss

    def fit(self, x_train, y_train, epochs=200, early_stopping_patience=5,
            batch_size=128, validation_split=0.1, train_shuffle=False):
        """
        fit train data
        """
        self.model = Sequential()
        self.model.add(LSTM(units=self.units, input_shape=(x_train.shape[1], x_train.shape[2])))
        self.model.add(Dropout(self.dropout_rate))
        self.model.add(Dense(units=y_train.shape[1], activation=self.activation))
        self.model.compile(optimizer=self.optimizer, loss=self.loss, metrics=["acc"])

        earlystopping = EarlyStopping(monitor="val_loss", patience=early_stopping_patience)
        self.model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
                       validation_split=validation_split, shuffle=train_shuffle,
                       callbacks=[earlystopping])
        
    def predict(self, x_test):
        """
        return predicted values of x_test
        """
        y_prob = self.model.predict(x_test)
        return y_prob

In [9]:
# parameters at model definition
def_params = {
    "units":100,
    "dropout_rate":0.2,
    "activation":"sigmoid",
    "optimizer":"adam",
    "loss":"binary_crossentropy"
}

# parameters at model training
fit_params = {
    "epochs":200,
    "early_stopping_patience":5,
    "batch_size":128,
    "validation_split":0.1,
    "train_shuffle":False
}

# number of time series cv splits
n_splits = 5

In [10]:
# set seed
np.random.seed(0)
os.environ["PYTHONHASHSEED"] = "0"
rn.seed(0)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
tf.set_random_seed(0)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

# define model
model = MyLSTM(**def_params)

# time series cross validation
train_scores_df, test_scores_df, y_test_probs = time_series_cv(x_all,
                                                               y_all,
                                                               n_splits=n_splits,
                                                               model=model,
                                                               fit_params=fit_params,
                                                               baseline=False)

---------- split 0 ----------
[17:55:42] train_index:0~3099 test_index:3100~6199
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 2789 samples, validate on 310 samples
Epoch 1/200
2789/2789 [==============================] - 6s 2ms/step - loss: 0.6396 - acc: 0.6466 - val_loss: 0.5360 - val_acc: 0.8287
Epoch 2/200
2789/2789 [==============================] - 5s 2ms/step - loss: 0.5208 - acc: 0.8284 - val_loss: 0.4194 - val_acc: 0.9378
Epoch 3/200
2789/2789 [==============================] - 5s 2ms/step - loss: 0.4456 - acc: 0.8798 - val_loss: 0.3453 - val_acc: 0.9525
Epoch 4/200
2789/2789 [==============================] - 5s 2ms/step - loss: 0.4003 - acc: 0.8862 - val_loss: 0.2981 - val_acc: 0.9525
Epoch 5/200
2789/2789 [==============================] - 5s 2ms/step - loss: 0.3729 - acc: 0.8866

5579/5579 [==============================] - 12s 2ms/step - loss: 0.2864 - acc: 0.9041 - val_loss: 0.2379 - val_acc: 0.9254
Epoch 12/200
5579/5579 [==============================] - 12s 2ms/step - loss: 0.2863 - acc: 0.9042 - val_loss: 0.2377 - val_acc: 0.9254
Epoch 13/200
5579/5579 [==============================] - 10s 2ms/step - loss: 0.2863 - acc: 0.9042 - val_loss: 0.2375 - val_acc: 0.9254
Epoch 14/200
5579/5579 [==============================] - 10s 2ms/step - loss: 0.2863 - acc: 0.9042 - val_loss: 0.2374 - val_acc: 0.9254
Epoch 15/200
5579/5579 [==============================] - 10s 2ms/step - loss: 0.2860 - acc: 0.9042 - val_loss: 0.2373 - val_acc: 0.9254
Epoch 16/200
5579/5579 [==============================] - 10s 2ms/step - loss: 0.2860 - acc: 0.9042 - val_loss: 0.2372 - val_acc: 0.9254
Epoch 17/200
5579/5579 [==============================] - 10s 2ms/step - loss: 0.2861 - acc: 0.9042 - val_loss: 0.2372 - val_acc: 0.9254
Epoch 18/200
5579/5579 [==============================

Epoch 15/200
11159/11159 [==============================] - 22s 2ms/step - loss: 0.2740 - acc: 0.9095 - val_loss: 0.2605 - val_acc: 0.9141
Epoch 16/200
11159/11159 [==============================] - 24s 2ms/step - loss: 0.2742 - acc: 0.9095 - val_loss: 0.2605 - val_acc: 0.9141
Epoch 17/200
11159/11159 [==============================] - 21s 2ms/step - loss: 0.2739 - acc: 0.9095 - val_loss: 0.2605 - val_acc: 0.9141
Epoch 18/200
11159/11159 [==============================] - 23s 2ms/step - loss: 0.2739 - acc: 0.9095 - val_loss: 0.2605 - val_acc: 0.9141
Epoch 19/200
11159/11159 [==============================] - 21s 2ms/step - loss: 0.2741 - acc: 0.9095 - val_loss: 0.2605 - val_acc: 0.9141
Epoch 20/200
11159/11159 [==============================] - 23s 2ms/step - loss: 0.2741 - acc: 0.9095 - val_loss: 0.2605 - val_acc: 0.9141
Epoch 21/200
11159/11159 [==============================] - 24s 2ms/step - loss: 0.2740 - acc: 0.9095 - val_loss: 0.2605 - val_acc: 0.9141
Epoch 22/200
11159/11159 [=

# Result

In [11]:
# train metrics
train_scores_df

,acc,log_loss
0,0.890226,0.306272
1,0.905388,0.278039
2,0.909121,0.270360
3,0.909125,0.270024
4,0.907303,0.273181


In [12]:
# test metrics
test_scores_df

,acc,log_loss
0,0.920532,0.256376
1,0.916590,0.254702
2,0.909153,0.269534
3,0.899998,0.286568
4,0.894838,0.298754


In [22]:
# predicted probability of crime occurance
y_probs_df = pd.DataFrame(y_test_probs, index=y_datetime, columns=geo_ids)
y_probs_df = y_probs_df.loc[(y_probs_df != 0).any(axis=1), :]
y_probs_df.head()

,06075010100,06075010200,06075010300,06075010400,06075010500,06075010600,06075010700,06075010800,06075010900,06075011000,06075011100,06075011200,06075011300,06075011700,06075011800,06075011901,06075011902,06075012000,06075012100,06075012201,06075012202,06075012301,06075012302,06075012401,06075012402,06075012501,06075012502,06075012601,06075012602,06075012700,06075012800,06075012901,06075012902,06075013000,06075013101,06075013102,06075013200,06075013300,06075013400,06075013500,06075015100,06075015200,06075015300,06075015400,06075015500,06075015600,06075015700,06075015801,06075015802,06075015900,06075016000,06075016100,06075016200,06075016300,06075016400,06075016500,06075016600,06075016700,06075016801,06075016802,06075016900,06075017000,06075017101,06075017102,06075017601,06075017700,06075017801,06075017802,06075017902,06075018000,06075020100,06075020200,06075020300,06075020401,06075020402,06075020500,06075020600,06075020700,06075020800,06075020900,06075021000,06075021100,06075021200,06075021300,06075021400,06075021500,06075021600,06075021700,06075021800,06075022600,06075022702,06075022704,06075022801,06075022802,06075022803,06075022901,06075022902,06075022903,06075023001,06075023003,06075023102,06075023103,06075023200,06075023300,06075023400,06075025100,06075025200,06075025300,06075025401,06075025402,06075025403,06075025500,06075025600,06075025701,06075025702,06075025800,06075025900,06075026001,06075026002,06075026003,06075026004,06075026100,06075026200,06075026301,06075026302,06075026303,06075026401,06075026402,06075026403,06075026404,06075030101,06075030102,06075030201,06075030202,06075030301,06075030302,06075030400,06075030500,06075030600,06075030700,06075030800,06075030900,06075031000,06075031100,06075031201,06075031202,06075031301,06075031302,06075031400,06075032601,06075032602,06075032700,06075032801,06075032802,06075032901,06075032902,06075033000,06075033100,06075033201,06075033203,06075033204,06075035100,06075035201,06075035202,06075035300,06075035400,06075040100,06075040200,06075042601,06075042602,06075042700,06075042800,06075045100,06075045200,06075047600,06075047701,06075047702,06075047801,06075047802,06075047901,06075047902,06075060100,06075060400,06075060502,06075060700,06075061000,06075061100,06075061200,06075061400,06075061500,06075980200,06075980300,06075980501,06075980600,06075980900
datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-09-17 08:00:00,0.389777,0.244693,0.088332,0.088769,0.295673,0.128599,0.184312,0.048144,0.093793,0.099397,0.201792,0.061619,0.070334,0.539163,0.047885,0.036579,0.053117,0.186075,0.153781,0.163472,0.218763,0.248866,0.180321,0.27775,0.449824,0.355478,0.225189,0.039057,0.094957,0.084847,0.176234,0.052589,0.088044,0.121513,0.061012,0.049833,0.077199,0.06214,0.048419,0.08622,0.146118,0.115873,0.055699,0.09067,0.162725,0.050579,0.167646,0.150002,0.082044,0.225033,0.107112,0.214192,0.192863,0.10065,0.13428,0.13367,0.214209,0.110886,0.090178,0.125199,0.159104,0.068377,0.043827,0.068613,0.642874,0.363733,0.185256,0.405793,0.057513,0.636138,0.440659,0.151721,0.131237,0.035381,0.044644,0.118726,0.169564,0.14497,0.220442,0.193297,0.063182,0.043723,0.026601,0.01726,0.04106,0.069483,0.040253,0.067703,0.062218,0.165388,0.064258,0.08159,0.144998,0.215158,0.075068,0.084417,0.044538,0.093259,0.078696,0.075653,0.103281,0.146762,0.186727,0.11654,0.128761,0.07907,0.109451,0.159243,0.071396,0.043155,0.085121,0.178149,0.067176,0.078185,0.105088,0.036007,0.060331,0.118393,0.04605,0.049366,0.051012,0.144502,0.076137,0.09062,0.057452,0.03131,0.054722,0.041544,0.060642,0.033944,0.061231,0.054722,0.068137,0.079247,0.063658,0.043157,0.055338,0.04156,0.030579,0.060933,0.10299,0.123768,0.066932,0.108986,0.083046,0.043232,0.054192,0.068924,0.080668,0.062629,0.035804,0.054999,0.079007,0.045583,0.042452,0.038851,0.119015,0.056145,0.122451,0.054178,0.056855,0.06